# PyRake-Style Optimization Examples

This notebook demonstrates the use of PyRake-style constrained optimization across IPW, G-computation, and AIPW estimators.

PyRake optimization simultaneously addresses:
- **Bias Reduction**: Covariate balance constraints
- **Variance Minimization**: Weight variance constraints
- **Predictive Power**: Minimal distance from baseline weights

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from causal_inference.core.base import CovariateData, OutcomeData, TreatmentData
from causal_inference.core.optimization_config import OptimizationConfig
from causal_inference.estimators.aipw import AIPWEstimator
from causal_inference.estimators.g_computation import GComputationEstimator
from causal_inference.estimators.ipw import IPWEstimator

# Set random seed for reproducibility
np.random.seed(42)

## Generate Synthetic Data

We'll create synthetic data with a known treatment effect of 2.0 and confounding.

In [ ]:
n = 1000
true_ate = 2.0

# Covariates
X = np.random.randn(n, 3)

# Treatment with confounding
propensity = 1 / (1 + np.exp(-(X[:, 0] + 0.5 * X[:, 1])))
treatment = np.random.binomial(1, propensity)

# Outcome with treatment effect = 2.0
outcome = (
    2.0 * treatment + X[:, 0] + 0.5 * X[:, 1] + 0.3 * X[:, 2] + np.random.randn(n) * 0.5
)

# Create data objects
treatment_data = TreatmentData(values=treatment, treatment_type="binary")
outcome_data = OutcomeData(values=outcome, outcome_type="continuous")
covariate_data = CovariateData(values=X, names=["X1", "X2", "X3"])

print(f"Sample size: {n}")
print(f"True ATE: {true_ate}")
print(f"Treatment prevalence: {np.mean(treatment):.2f}")

## IPW with Weight Optimization

Compare standard IPW with optimized weights that minimize variance while maintaining covariate balance.

In [ ]:
# Standard IPW
print("=" * 60)
print("STANDARD IPW")
print("=" * 60)
estimator_standard = IPWEstimator(
    propensity_model_type="logistic", random_state=42, verbose=True
)
estimator_standard.fit(treatment_data, outcome_data, covariate_data)
effect_standard = estimator_standard.estimate_ate()

print(f"\nATE Estimate: {effect_standard.ate:.4f}")
print(f"Bias: {abs(effect_standard.ate - true_ate):.4f}")

In [ ]:
# Optimized IPW
print("\n" + "=" * 60)
print("OPTIMIZED IPW")
print("=" * 60)
optimization_config = OptimizationConfig(
    optimize_weights=True,
    variance_constraint=2.0,
    balance_constraints=True,
    balance_tolerance=0.01,
    distance_metric="l2",
    verbose=True,
)

estimator_optimized = IPWEstimator(
    propensity_model_type="logistic",
    optimization_config=optimization_config,
    random_state=42,
    verbose=True,
)
estimator_optimized.fit(treatment_data, outcome_data, covariate_data)
effect_optimized = estimator_optimized.estimate_ate()

print(f"\nATE Estimate: {effect_optimized.ate:.4f}")
print(f"Bias: {abs(effect_optimized.ate - true_ate):.4f}")

In [ ]:
# Compare weight distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(estimator_standard.weights, bins=50, alpha=0.7, edgecolor="black")
axes[0].set_title("Standard IPW Weights")
axes[0].set_xlabel("Weight")
axes[0].set_ylabel("Frequency")

axes[1].hist(
    estimator_optimized.weights, bins=50, alpha=0.7, color="orange", edgecolor="black"
)
axes[1].set_title("Optimized IPW Weights")
axes[1].set_xlabel("Weight")
axes[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

# Print diagnostics comparison
print("\nWeight Diagnostics Comparison:")
print("-" * 60)
print(f"{'Metric':<30} {'Standard':<15} {'Optimized':<15}")
print("-" * 60)
for key in estimator_standard._weight_diagnostics:
    std_val = estimator_standard._weight_diagnostics[key]
    opt_val = estimator_optimized._weight_diagnostics[key]
    print(f"{key:<30} {std_val:<15.4f} {opt_val:<15.4f}")

## G-Computation with Ensemble Models

Use ensemble of multiple models with optimized weights for improved prediction.

In [ ]:
# Single model G-computation
print("=" * 60)
print("SINGLE MODEL G-COMPUTATION")
print("=" * 60)
estimator_single = GComputationEstimator(
    model_type="linear", random_state=42, verbose=True
)
estimator_single.fit(treatment_data, outcome_data, covariate_data)
effect_single = estimator_single.estimate_ate()

print(f"\nATE Estimate: {effect_single.ate:.4f}")
print(f"Bias: {abs(effect_single.ate - true_ate):.4f}")

In [ ]:
# Ensemble G-computation
print("\n" + "=" * 60)
print("ENSEMBLE G-COMPUTATION")
print("=" * 60)
estimator_ensemble = GComputationEstimator(
    use_ensemble=True,
    ensemble_models=["linear", "ridge", "random_forest"],
    ensemble_variance_penalty=0.1,
    random_state=42,
    verbose=True,
)
estimator_ensemble.fit(treatment_data, outcome_data, covariate_data)
effect_ensemble = estimator_ensemble.estimate_ate()

print(f"\nATE Estimate: {effect_ensemble.ate:.4f}")
print(f"Bias: {abs(effect_ensemble.ate - true_ate):.4f}")

In [ ]:
# Visualize ensemble weights
if estimator_ensemble.ensemble_weights is not None:
    model_names = list(estimator_ensemble.ensemble_models_fitted.keys())
    weights = estimator_ensemble.ensemble_weights

    plt.figure(figsize=(8, 5))
    plt.bar(model_names, weights, color="steelblue", edgecolor="black")
    plt.xlabel("Model")
    plt.ylabel("Weight")
    plt.title("Optimized Ensemble Weights")
    plt.ylim(0, 1)
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

    print("\nEnsemble Weight Distribution:")
    for name, weight in zip(model_names, weights):
        print(f"  {name}: {weight:.4f}")

## AIPW with Component Balance Optimization

Optimize the balance between G-computation and IPW components in the doubly-robust estimator.

In [ ]:
# Standard AIPW
print("=" * 60)
print("STANDARD AIPW")
print("=" * 60)
estimator_aipw_standard = AIPWEstimator(
    cross_fitting=True, n_folds=3, random_state=42, verbose=True
)
estimator_aipw_standard.fit(treatment_data, outcome_data, covariate_data)
effect_aipw_standard = estimator_aipw_standard.estimate_ate()

print(f"\nATE Estimate: {effect_aipw_standard.ate:.4f}")
print(f"Bias: {abs(effect_aipw_standard.ate - true_ate):.4f}")

In [ ]:
# Optimized AIPW
print("\n" + "=" * 60)
print("OPTIMIZED AIPW")
print("=" * 60)
estimator_aipw_optimized = AIPWEstimator(
    cross_fitting=True,
    n_folds=3,
    optimize_component_balance=True,
    component_variance_penalty=0.5,
    random_state=42,
    verbose=True,
)
estimator_aipw_optimized.fit(treatment_data, outcome_data, covariate_data)
effect_aipw_optimized = estimator_aipw_optimized.estimate_ate()

print(f"\nATE Estimate: {effect_aipw_optimized.ate:.4f}")
print(f"Bias: {abs(effect_aipw_optimized.ate - true_ate):.4f}")

In [ ]:
# Compare component contributions
opt_diag = estimator_aipw_optimized.get_optimization_diagnostics()

if opt_diag is not None:
    print("\nComponent Balance Optimization Results:")
    print("-" * 60)
    print(
        f"Optimal G-computation weight: {opt_diag['optimal_g_computation_weight']:.4f}"
    )
    print(f"Optimal IPW weight: {opt_diag['optimal_ipw_weight']:.4f}")
    print(f"Variance (fixed 50/50): {opt_diag['fixed_variance']:.6f}")
    print(f"Variance (optimized): {opt_diag['optimized_variance']:.6f}")
    print(
        f"Variance reduction: {opt_diag['fixed_variance'] - opt_diag['optimized_variance']:.6f}"
    )

    # Visualize component weights
    components = ["G-computation", "IPW"]
    weights = [opt_diag["optimal_g_computation_weight"], opt_diag["optimal_ipw_weight"]]

    plt.figure(figsize=(8, 5))
    plt.bar(components, weights, color=["steelblue", "orange"], edgecolor="black")
    plt.axhline(
        y=0.5, color="red", linestyle="--", label="Equal weights (standard AIPW)"
    )
    plt.xlabel("Component")
    plt.ylabel("Weight")
    plt.title("Optimized AIPW Component Weights")
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

## Bias-Variance Tradeoff Exploration

Explore how varying the variance constraint affects the bias-variance tradeoff in IPW.

In [ ]:
# Test different variance constraints
variance_constraints = [1.0, 1.5, 2.0, 2.5, 3.0, None]
results = []

print("Testing different variance constraints...\n")

for vc in variance_constraints:
    config = OptimizationConfig(
        optimize_weights=True,
        variance_constraint=vc,
        balance_constraints=True,
        distance_metric="l2",
        verbose=False,
    )

    estimator = IPWEstimator(
        propensity_model_type="logistic",
        optimization_config=config,
        random_state=42,
        verbose=False,
    )
    estimator.fit(treatment_data, outcome_data, covariate_data)
    effect = estimator.estimate_ate()

    results.append(
        {
            "variance_constraint": vc if vc is not None else "None",
            "ate": effect.ate,
            "bias": abs(effect.ate - true_ate),
            "weight_variance": estimator._weight_diagnostics["weight_variance"],
            "ess": estimator._weight_diagnostics["effective_sample_size"],
        }
    )

    vc_str = f"{vc:.1f}" if vc is not None else "None"
    print(
        f"Variance constraint: {vc_str:>6} | "
        f"ATE: {effect.ate:6.3f} | "
        f"Bias: {abs(effect.ate - true_ate):6.4f} | "
        f"Weight Var: {estimator._weight_diagnostics['weight_variance']:6.4f} | "
        f"ESS: {estimator._weight_diagnostics['effective_sample_size']:7.1f}"
    )

In [ ]:
# Visualize bias-variance tradeoff
import pandas as pd

df_results = pd.DataFrame(results)
df_results_numeric = df_results[df_results["variance_constraint"] != "None"].copy()
df_results_numeric["variance_constraint"] = df_results_numeric[
    "variance_constraint"
].astype(float)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bias vs variance constraint
axes[0].plot(
    df_results_numeric["variance_constraint"],
    df_results_numeric["bias"],
    marker="o",
    linewidth=2,
    markersize=8,
    color="steelblue",
)
axes[0].set_xlabel("Variance Constraint")
axes[0].set_ylabel("Absolute Bias")
axes[0].set_title("Bias vs Variance Constraint")
axes[0].grid(alpha=0.3)

# Weight variance vs variance constraint
axes[1].plot(
    df_results_numeric["variance_constraint"],
    df_results_numeric["weight_variance"],
    marker="o",
    linewidth=2,
    markersize=8,
    color="orange",
)
axes[1].set_xlabel("Variance Constraint")
axes[1].set_ylabel("Weight Variance")
axes[1].set_title("Weight Variance vs Constraint")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **IPW Optimization**: Reducing weight variance while maintaining covariate balance
2. **G-Computation Ensembles**: Using multiple models with optimized weights
3. **AIPW Component Balance**: Optimizing the contribution of G-computation and IPW components
4. **Bias-Variance Tradeoff**: Exploring the impact of variance constraints

Key takeaways:
- Optimization can reduce variance without introducing bias
- Ensemble methods provide robustness across different model specifications
- Component balance in AIPW adapts to data characteristics
- Variance constraints allow fine-tuning the bias-variance tradeoff